In [1]:
import numpy as np
from scipy import stats
from scipy.stats import skew
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.externals import joblib   # 함수는 dump 시켜도 안됨
from sklearn.feature_selection import SelectPercentile, f_classif, f_regression, SelectFromModel, RFE
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [3]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

In [4]:
from my_transformer import change_to_str, divide_columns, feature_selection, simple_imputer, one_hot_encoding, concat, rf_imputer, fill_columns
from my_transformer import rmsle_scorer, neg_rmsle_scorer, rmsle

In [5]:
def preparation(data) :
    
    data.drop(['SalePrice', 'Id'], axis=1, inplace=True)
    
    astype_str = change_to_str('MSSubClass')
    data = astype_str.fit_transform(data)
          
    data.drop(['Alley', 'Fence', 'FireplaceQu', 'MiscFeature', 'PoolQC'], axis=1, inplace=True)
    
    num_columns, cat_columns = divide_columns(data)
    
    pipeline_cat = make_pipeline(
        feature_selection(cat_columns),
#         simple_imputer('most_frequent'),
        one_hot_encoding(cat_columns)
    )
    
    X_cat = pipeline_cat.fit_transform(data)
    X_num = data[num_columns]
    X = concat(X_num, X_cat)
    
    isnull_sum = X.isnull().sum()
    not_null = list(isnull_sum[isnull_sum == 0].index)
    null_columns = list(isnull_sum[isnull_sum > 0])
    
    for column in num_columns :
        X = fill_columns(X, column, 'mean')
        
    isnull_sum = X.isnull().sum()
    print(isnull_sum[isnull_sum > 0].sort_values(ascending=False))
    
    skew_features = X[num_columns].apply(lambda x : skew(x))
    skew_features_top = skew_features[skew_features > 1]
    X[skew_features_top.index] = np.log1p(X[skew_features_top.index])
            
    return X

In [6]:
def data_set() :
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
    
    data = pd.concat([train, test], axis=0)
    
    X = preparation(data)
    X_train = X.iloc[:1460, :]
    X_test = X.iloc[1460:, :]
    
    y = train['SalePrice']
    y_train = np.log1p(y)
    
    return X_train, X_test, y_train

In [7]:
def remove_outlier(column) :
    index_outlier = X_train[abs(X_train[column] - X_train[column].mean()) > (3 * X_train[column].std())].index
    print(len(index_outlier))
    X_train.drop(index=index_outlier, axis=0, inplace=True)
    y_train.drop(index=index_outlier, axis=0, inplace=True)
    print(X_train.shape)

In [8]:
X_train, X_test, y_train = data_set()

Series([], dtype: int64)


In [9]:
remove_outlier('GrLivArea')

8
(1452, 285)


In [10]:
ridge = Ridge(alpha=10, random_state=30)
lasso = Lasso(alpha=0.001, random_state=30)
elastic = ElasticNet(alpha=0.001, l1_ratio=0.5, random_state=30)
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=30)
xgb = XGBRegressor(random_state=30)

In [11]:
models = {
    'ridge' : ridge,
    'lasso' : lasso,
    'elastic' : elastic
}

In [12]:
scores = dict()

for key, model in models.items() : 
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()   
    scores[key] = round(np.sqrt(-score), 3)
    
scores

{'ridge': 0.113, 'lasso': 0.112, 'elastic': 0.11}

### Submission

In [13]:
for key, model in models.items() :
    score = scores[key]
    
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    prediction = np.exp(prediction)
    
    submission = pd.read_csv("sample_submission.csv")
    submission["SalePrice"] = prediction
    submission.to_csv("score_{}_{:.3f}_submission.csv".format(key, score), index=False)

## # Mix

In [14]:
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train, y_train, random_state=30)

ridge.fit(X_train_split, y_train_split)
lasso.fit(X_train_split, y_train_split)
elastic.fit(X_train_split, y_train_split)

y_rideg = ridge.predict(X_test_split)
y_lasso = lasso.predict(X_test_split)
y_elestic = elastic.predict(X_test_split)

In [15]:
predict = {
    'r_4_l_6' : 0.4 * y_rideg + 0.6 * y_lasso,
    'l_4_e_6' : 0.4 * y_lasso + 0.6 * y_elestic,
    'l_6_e_4' : 0.6 * y_lasso + 0.4 * y_elestic,
    'l_8_e_2' : 0.8 * y_lasso + 0.2 * y_elestic,
}

In [16]:
scores = dict()

def rmse(dic) :
    for key, value in dic.items() :
        mse = mean_squared_error(y_test_split, value)
        rmse = np.sqrt(mse)
        
        scores[key] = rmse

In [17]:
rmse(predict)
scores

{'r_4_l_6': 0.11766882639179346,
 'l_4_e_6': 0.11746850322207804,
 'l_6_e_4': 0.11739662085858443,
 'l_8_e_2': 0.11744174909977348}

In [18]:
ridge.fit(X_train, y_train)
lasso.fit(X_train, y_train)
elastic.fit(X_train, y_train)

y_rideg = ridge.predict(X_test)
y_lasso = lasso.predict(X_test)
y_elestic = elastic.predict(X_test)

In [19]:
predict = {
    'r_4_l_6' : 0.4 * y_rideg + 0.6 * y_lasso,
    'l_4_e_6' : 0.4 * y_lasso + 0.6 * y_elestic,
    'l_6_e_4' : 0.6 * y_lasso + 0.4 * y_elestic,
    'l_8_e_2' : 0.8 * y_lasso + 0.2 * y_elestic,
}

### Submission

In [20]:
for key, pred in predict.items() :
    score = scores[key]

    prediction = np.exp(pred)
    
    submission = pd.read_csv("sample_submission.csv")
    submission["SalePrice"] = prediction
    submission.to_csv("score_{}_{:.3f}_submission.csv".format(key, score), index=False)

## # stacking

In [21]:
# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수. 
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=0)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n[train_index] 
        y_tr = y_train_n[train_index] 
        X_te = X_train_n[valid_index]
        
        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)       
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
        test_pred[:, folder_counter] = model.predict(X_test_n)
            
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean

In [22]:
# get_stacking_base_datasets( )은 넘파이 ndarray를 인자로 사용하므로 DataFrame을 넘파이로 변환. 
X_train_n = X_train_split.values
X_test_n = X_test_split.values
y_train_n = y_train_split.values

# 각 개별 기반(Base)모델이 생성한 학습용/테스트용 데이터 반환. 
ridge_train, ridge_test = get_stacking_base_datasets(ridge, X_train_n, y_train_n, X_test_n, 5)
elastic_train, elastic_test = get_stacking_base_datasets(elastic, X_train_n, y_train_n, X_test_n, 5)
gb_train, gb_test = get_stacking_base_datasets(gb, X_train_n, y_train_n, X_test_n, 5)
xgb_train, xgb_test = get_stacking_base_datasets(xgb, X_train_n, y_train_n, X_test_n, 5)  

Ridge  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
ElasticNet  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
GradientBoostingRegressor  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
XGBRegressor  model 시작 
	 폴드 세트:  0  시작 
[20:36:07] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  1  시작 
[20:36:09] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  2  시작 
[20:36:10] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  3  시작 
[20:36:12] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/reg

In [23]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 Stacking 형태로 결합.  
Stack_final_X_train = np.concatenate((ridge_train, elastic_train, gb_train, xgb_train), axis=1)
Stack_final_X_test = np.concatenate((ridge_test, elastic_test, gb_test, xgb_test), axis=1)

# 최종 메타 모델은 라쏘 모델을 적용. 
meta_model_lasso = Lasso(alpha=0.001, random_state=30)

#기반 모델의 예측값을 기반으로 새롭게 만들어진 학습 및 테스트용 데이터로 예측하고 RMSE 측정.
meta_model_lasso.fit(Stack_final_X_train, y_train_split)
final = meta_model_lasso.predict(Stack_final_X_test)
mse = mean_squared_error(y_test_split , final)
rmse = np.sqrt(mse)
print('스태킹 회귀 모델의 최종 RMSE 값은:', round(rmse, 3))

스태킹 회귀 모델의 최종 RMSE 값은: 0.116


In [24]:
score = round(rmse, 3)

In [25]:
# get_stacking_base_datasets( )은 넘파이 ndarray를 인자로 사용하므로 DataFrame을 넘파이로 변환. 
X_train_n = X_train.values
X_test_n = X_test.values
y_train_n = y_train.values

# 각 개별 기반(Base)모델이 생성한 학습용/테스트용 데이터 반환. 
ridge_train, ridge_test = get_stacking_base_datasets(ridge, X_train_n, y_train_n, X_test_n, 5)
elastic_train, elastic_test = get_stacking_base_datasets(elastic, X_train_n, y_train_n, X_test_n, 5)
gb_train, gb_test = get_stacking_base_datasets(gb, X_train_n, y_train_n, X_test_n, 5)
xgb_train, xgb_test = get_stacking_base_datasets(xgb, X_train_n, y_train_n, X_test_n, 5)  

Ridge  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
ElasticNet  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
GradientBoostingRegressor  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
XGBRegressor  model 시작 
	 폴드 세트:  0  시작 
[20:36:25] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  1  시작 
[20:36:28] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  2  시작 
[20:36:31] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  3  시작 
[20:36:33] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/reg

In [26]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 Stacking 형태로 결합.  
Stack_final_X_train = np.concatenate((ridge_train, elastic_train, gb_train, xgb_train), axis=1)
Stack_final_X_test = np.concatenate((ridge_test, elastic_test, gb_test, xgb_test), axis=1)

# 최종 메타 모델은 라쏘 모델을 적용. 
meta_model_lasso = Lasso(alpha=0.001, random_state=30)

#기반 모델의 예측값을 기반으로 새롭게 만들어진 학습 및 테스트용 데이터로 예측하고 RMSE 측정.
meta_model_lasso.fit(Stack_final_X_train, y_train)
prediction = meta_model_lasso.predict(Stack_final_X_test)

In [27]:
prediction = np.exp(pred)
    
submission = pd.read_csv("sample_submission.csv")
submission["SalePrice"] = prediction
submission.to_csv("score_{}_{:.3f}_submission.csv".format('stack', score), index=False)